# Librairies

In [ ]:
import pandas as pd
import numpy as np
from datetime import date
import os
import json
from natsort import natsorted, ns
import re 
import itertools
import umap.umap_ as umap
import hdbscan
import pubchempy
import time
import multiprocessing

# RDkit
import rdkit
from rdkit import Chem, DataStructs

# Plotting
import matplotlib.colors as colors
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

# Scikit-learn
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.neighbors import KernelDensity
from sklearn.metrics import pairwise_distances

# Functions

In [ ]:
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

def get_family(x):
    res = re.search(r'^OR([0-9]+)[A-Z]+[0-9]+$',x)[1]
    return res

def compute_smiles(cids_list, batch_size, properties='smiles'):
    def batch(iterable, n=1):
        l = len(iterable)
        for ndx in range(0, l, n):
            yield iterable[ndx:min(ndx + n, l)]
    
    _smiles = []
    for x in batch(range(0, len(cids_list)), batch_size):
        batch_cid = [cids_list[i] for i in x]
        if properties == 'smiles':
            batch_smiles = [i.isomeric_smiles for i in pubchempy.get_compounds(batch_cid, namespace='inchikey')]
        elif properties == 'inchikey':
            batch_smiles = [i.inchikey for i in pubchempy.get_compounds(batch_cid)]
        _smiles.append(batch_smiles)
    
    return list(itertools.chain(*_smiles))

def _change_units(x, _from, _to):
    try:
        x = float(x)
    except:
        return x
    if _from == 'uM' and _to == 'Log(M)':
        return np.log10(x) - 6.0 # uM = 10^-6 M
    elif _from == 'Log(M)' and _to == 'uM':
        return 10.0**(x + 6.0)
    elif _from == 'mM' and _to == 'uM':
        return x*(10.0**3)

def _func_data_ec50(data_ec50):
    pairs_ec50 = data_ec50.groupby(['_MolID', 'mutated_Sequence']).apply(lambda x: x['Responsive'].mean()) # TODO: apply more complex function returning {'Responsive', 'sample_weight'}
    if ((pairs_ec50 > 0.0)&(pairs_ec50 < 1.0)).any():
        _ec50_inconsitent_idx = pairs_ec50[((pairs_ec50 > 0.0)&(pairs_ec50 < 1.0))].index
        print('INFO: To discard EC50 inconsistent: {}'.format(len(_ec50_inconsitent_idx)))
        pairs_ec50 = pairs_ec50.loc[pairs_ec50.index.difference(_ec50_inconsitent_idx)]
    pairs_ec50 = pairs_ec50.astype(int)
    pairs_ec50.name = 'Responsive'
    pairs_ec50 = pairs_ec50.to_frame()
    pairs_ec50['_DataQuality'] = 'ec50'
    return pairs_ec50

def _func_data_screening(data_screening):
    """
    """
    def num_unique_value_screen(_df):
        return len(_df['Value_Screen'].unique())

    def _is_sorted(s):
        return all(s.iloc[i] <= s.iloc[i+1] for i in range(len(s) - 1))
        
    _screening_consistency_ordering = data_screening.groupby(['_MolID', 'mutated_Sequence']).apply(lambda x: _is_sorted(x.sort_values(['Value_Screen', 'Responsive'])['Responsive']))
    _screening_consistency_ordering.name = 'Check'
    _screening_inconsistent_ordering_idx = _screening_consistency_ordering[~_screening_consistency_ordering].index
    _screening_consistent_ordering_idx = _screening_consistency_ordering[_screening_consistency_ordering].index
    print('INFO: To discard because of screening ordering (Inconsistent through Value_Screen): {}'.format(len(_screening_inconsistent_ordering_idx)))

    _screening_consistency_per_value = data_screening.groupby(['_MolID', 'mutated_Sequence', 'Value_Screen']).apply(lambda x: ((x['Responsive']==1).all() or (x['Responsive']==0).all()))
    _screening_consistency_per_value.name = 'Check'
    _screening_consistency_per_value = _screening_consistency_per_value.reset_index('Value_Screen')
    _screening_inconsitent_per_value_idx = _screening_consistency_per_value[~_screening_consistency_per_value['Check']].index
    _screening_inconsitent_per_value_idx = _screening_inconsitent_per_value_idx.drop_duplicates()
    _screening_consitent_per_value_idx = _screening_consistency_per_value[_screening_consistency_per_value['Check']].index
    _screening_consitent_per_value_idx = _screening_consitent_per_value_idx.drop_duplicates()
    print('INFO: To discard screening inconsistent per Value_Screen: {}'.format(len(_screening_inconsitent_per_value_idx)))

    _screening_inconsitent_idx = _screening_inconsitent_per_value_idx.union(_screening_inconsistent_ordering_idx)
    print('INFO: To discard screening inconsistent: {}'.format(len(_screening_inconsitent_idx)))

    pairs_screening = data_screening.groupby(['_MolID', 'mutated_Sequence']).apply(lambda x: (x['Responsive']==1).any()).astype(int) # TODO: apply more complex function returning {'Responsive', 'sample_weight'}
    pairs_screening.name = 'Responsive'
    pairs_screening = pairs_screening.to_frame()
    pairs_screening = pairs_screening.loc[pairs_screening.index.difference(_screening_inconsitent_idx)] # TODO: Discarding inconsistent here. Do we want to do it?

    _count_unique_concentrations = data_screening.groupby(['_MolID', 'mutated_Sequence']).apply(num_unique_value_screen)
    _count_unique_concentrations.name = 'num_unique_value_screen'

    pairs_screening = pairs_screening.join(_count_unique_concentrations, how='left')        

    pairs_primary = pairs_screening[pairs_screening['num_unique_value_screen'] == 1].copy()
    pairs_primary['_DataQuality'] = 'primaryScreening'
    pairs_secondary = pairs_screening[pairs_screening['num_unique_value_screen'] > 1].copy()
    pairs_secondary['_DataQuality'] = 'secondaryScreening'

    return pairs_primary, pairs_secondary

def choose_smiles(x):
    """
    This is consistent with prepro.py
    """
    if x['GS_SMILES'] == x['GS_SMILES']:
        return x['GS_SMILES']
    else:
        return x['paper_SMILES']

def broadness_measure(x):
    n = x.shape[0]
    x_bool = x > 0.5
    # return x_bool
    return sum(x_bool).astype(np.float32) / n

def get_map_CID_to_inchikey(cid):
    mols = pubchempy.get_compounds(cid, 'cid')
    return {mol.cid: mol.inchikey for mol in mols if mol is not None}

def _unique_atomic_num(smiles):
    mol = Chem.MolFromSmiles(smiles.strip())
    X = []
    for atom in mol.GetAtoms():
        X.append(atom.GetAtomicNum())
    x = np.array(X, dtype = int)
    return np.unique(x)
    
def _test_CNOS_atomic_num(smiles):
    mol = Chem.MolFromSmiles(smiles.strip())
    X = []
    for atom in mol.GetAtoms():
        X.append(atom.GetAtomicNum())
    x = np.array(X, dtype = int)
    for ele in np.unique(x):
        if ele not in [6, 7, 8, 16]:
            return False
    return True

def _get_kernel_density_contour_plot(samples, _xgrid, _ygrid):
    X, Y = np.meshgrid(_xgrid, _ygrid)
    xy = np.vstack([X.ravel(), Y.ravel()]).T

    kde = KernelDensity(kernel="cosine")
    kde.fit(samples)

    Z = np.exp(kde.score_samples(xy))
    Z = Z.reshape(X.shape)

    start_level = 0.0013 + Z.min() 
    end_level = Z.max()
    levels = np.linspace(start_level, end_level, 10)

    return X, Y, Z, levels, kde

def _get_most_occuring_pyrfume_descriptor(df, i = 0, return_stats = False, top_k = 1):
    s = df['pyrfume_odor'].str.split(',').explode()
    _count = s.groupby(s).count().sort_values(ascending = False)
    if return_stats:
        return _count.index[i], _count.iloc[i], _count.iloc[i]/len(df)
    else:
        return '_'.join(_count.index[i:i+top_k].to_list())

def get_quantile_per_odor_group(df, q = 0.75):
    if len(df) > 9:
        _X = np.stack(df['predict'])
        _X = (_X > 0.5).astype(float)
        _dist = pairwise_distances(_X, metric = 'l1')
        try:
            _dist_triu = _dist[np.triu_indices(n = _dist.shape[0], k = 1)] # /(len(df) - 1)
            median_distances = np.median(_dist, axis=1)
            # nanmedian_distances = np.nanmedian(_dist, axis=1)
            # nanmedian_distances = np.sum(nanmedian_distances, axis=1) / (len(nanmedian_distances) - 1)  
            return np.quantile(median_distances, q = q)
        except Exception as e:
            print(df['predict'])
            raise e
    else:
        return float('nan')

def perform_mutation(row, mutation_col = 'Mutation', seq_col = 'Sequence'):
    """
    Perform mutation on a row in pandas.DataFrame. This function is designed to be used in pandas apply.

    Paramters:
    ----------
    row : pandas.Series
        row of pandas dataframe

    mutation_col : str
        name of the column with mutation information.

    seq_col : str
        name of the column with sequences.

    Returns:
    --------
    mutated_seq : str
        mutated sequence
    """
    try:
        if row[mutation_col] == row[mutation_col] and row[seq_col] == row[seq_col]:
            seq = row[seq_col]
            mutations = row[mutation_col].strip().split('_')
            return _perform_mutation(mutations, seq)
        else:
            return row[seq_col]
    except Exception as e:
        print(row)
        print(row[seq_col])
        raise  e

def _perform_mutation(mutations, seq):
    """
    Main logic for performing mutations.
    """    
    for mutation in mutations:
        # print('->' + mutation)
        _from = mutation[0]
        _to = mutation[-1]
        _position = int(mutation[1:-1]) - 1
        if seq[_position] == _from:
            seq = seq[:_position] + _to + seq[_position+1:]
        else:
            left_pos = _position - 5
            right_pos = _position + 4
            if _position < 5: left_pos = 0
            if _position > len(seq) - 4: right_pos = len(seq)
            raise 'Expected letter {} on position {} in sequence arround: {}. Found {}. Mutation: {}'.format(_from, _position, seq[left_pos:right_pos], seq[_position], mutation)
    return seq

def get_quantile_per_code_group(df, q = 0.75):
    if len(df) > 15:
        _X = np.stack(df['embed'])
        _dist = pairwise_distances(_X, metric = 'l2')
        try:
            _dist_triu = _dist[np.triu_indices(n = _dist.shape[0], k = 1)] # /(len(df) - 1)
            return np.quantile(_dist_triu, q = q)
        except Exception as e:
            print(df['embed'])
            raise e
    else:
        return float('nan')

def merge_cols_with_priority(row, primary_col = 'Uniprot_Sequence', secondary_col = 'Sequence'): # merge_seqs
    """
    merge two columns. primary column is kept and only if entry is missing use secondary_col.
    This should be used in pandas apply.

    Paramters:
    ----------
    row : pandas.Series
        row of pandas dataframe.
    
    primary_col : str
        name of the main column.

    secondary_col : str
        name of the secondary column used only when info in the first is missing.

    Returns:
    --------
    entry : Any
        value to put to the merged column.
    """
    if row[primary_col] == row[primary_col]:
        return row[primary_col]
    else:
        return row[secondary_col]

def nDscatterPlot_online(ax, X, labels, special_labels, special_formatting, legend, cmap = None):
    """
    Paramters:
    ----------
    X : numpy.ndarray
        array with (x, y) coordinates. It has to have shape [n_points, 2].

    labels : numpy.ndarray
        array with labels for each datapoint. It has to have the same first dimension as X.

    special_labels : list
        list of special labels, that can be annotated and have non-standard marker and properties.

    special_formatting : dict
        formatting of the special points. It is a dictionary with special_labels as keys and
        values are dictionaries for each label's formatting. Structure is as follows:
            {label_1 : {'marker' : 's',
                        'annotations' : some_array,
                        'color' : 0},
             label_2 : {'marker' : 'x'},
             label_3 : {'annotations' : some_array}}
        If there is some missing property, default is used for it.
        Currently availabel properties: ['marker', 'annotations']
    """
    if cmap is None:
        cmap = plt.cm.nipy_spectral
    # cmap = plt.get_cmap('hsv')
    if labels is None:
        labels = np.zeros(X.shape[0], dtype=np.int32)

    labels_unique = np.unique(labels)
    N = labels_unique.shape[0]
    if N == 1:
        N_eff = 1
    else:
        N_eff = N-1
    for i, lab_i in enumerate(labels_unique):
        c = cmap(float(i)/N_eff)
        idx = np.squeeze(np.argwhere(labels==lab_i))
        _X = [X[idx,i] for i in range(X.shape[1])]
        if lab_i in special_labels:
            formatting = special_formatting.get(lab_i, {'marker' : 'x',
                                                        'annotations': [],
                                                        'color' : 0,
                                                        'size' : 6,
                                                        'lw': 0,
                                                        'lc':[(0,0,0,1)]})
            marker = formatting.get('marker', 'x')
            annos = formatting.get('annotations', [])
            cmap_idx = formatting.get('color', 0)
            size_coef = formatting.get('size', 6)
            lw = formatting.get('lw', 0)
            lc = formatting.get('lc', [(0,0,0,1)])
            if len(idx.shape) > 0:
                assert len(annos) == idx.shape[0] or len(annos)==0
            else:
                assert len(annos)==0
            ax.scatter(*_X, color=cmap(cmap_idx), label=lab_i, marker=marker, s=size_coef*(plt.rcParams['lines.markersize'])**2, linewidths=lw, edgecolors=lc)
            for _i, txt in enumerate(annos):
                ax.text(*[_X[k].item(_i) + 0.3 for k in range(X.shape[1])], txt)
        else:
            ax.scatter(*_X, color=c, label=lab_i)
    xlim = ax.get_xlim()
    ax.set_xlim((xlim[0], xlim[1]+3))
    if legend:
        ax.legend(loc="upper right")

def ORs_in_contour_plot(ax, _df_embed_joint, _embedding_name, _gene_names):
    _X_all = np.stack(_df_embed_joint[_embedding_name])
    _x_min, _y_min = np.min(_X_all, axis = 0)
    _x_max, _y_max = np.max(_X_all, axis = 0)
    _xgrid = np.arange(start = _x_min - 1, stop = _x_max + 1, step = 0.2)
    _ygrid = np.arange(start = _y_min - 1, stop = _y_max + 1, step = 0.2)

    _contour_plots = {}
    for i, name in enumerate(_gene_names):
        _tmp = df_predict_unstack[name]
        _idx = _tmp[_tmp > 0.5].index.intersection(_df_embed_joint.index)
        _s = _df_embed_joint.loc[_idx][_embedding_name]

        _X = np.stack(_s)
        X, Y, Z, levels, _  = _get_kernel_density_contour_plot(_X, _xgrid, _ygrid)

        #_cmap = plt.cm.get_cmap('Blues')
        _cmap = plt.cm.get_cmap('viridis')
        newcolors = np.array(_cmap(np.linspace(0, 1, 256))) + np.array([[100*i/256, 3*i/256, 3*i/256, 0]])
        newcolors = list((newcolors - np.min(newcolors, axis = 0))/np.max(newcolors, axis = 0))
        newcmp = ListedColormap(newcolors)

        def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=256):
            new_cmap = colors.LinearSegmentedColormap.from_list(
                'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
                cmap(np.linspace(minval, maxval, n)))  
            return new_cmap

        greys = truncate_colormap(plt.cm.get_cmap('Greys'),0.4,1)
        reds = truncate_colormap(plt.cm.get_cmap('Greens'),0.4,1)
        blues = truncate_colormap(plt.cm.get_cmap('Blues'),0.4,1)
        colormaplist = [greys, reds, blues]
        newcmp = colormaplist[i]

        _contour_plots[name] = ax.contourf(X, Y, Z, levels=levels, cmap=newcmp, alpha = 0.45)
        _pos_max = np.where(Z == np.max(Z))

        _dummy_point = ax.plot(X[_pos_max], Y[_pos_max], label = name, color=newcolors[0][:3]) # Create a dummy point

        del _tmp, _idx, _s, _X, newcmp, newcolors

    
    def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=256):
        new_cmap = colors.LinearSegmentedColormap.from_list(
            'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
            cmap(np.linspace(minval, maxval, n)))  
        return new_cmap
    a = truncate_colormap(plt.get_cmap('copper'),.5,1)(np.linspace(0., 1, 128))
    b = truncate_colormap(plt.get_cmap('jet'),0.8,1)(np.linspace(0., 1, 128))
    couleur = np.vstack((a, b,[0., 0., 0, 1.]))
    mymap = LinearSegmentedColormap.from_list('my_colormap', couleur)

    # plots.nDscatterPlot(ax, X = _X_all, labels = np.stack(_df_embed_joint['GS_Odor type']), special_labels = [], special_formatting = {}, legend = True, cmap = None)
    a = nDscatterPlot_online(ax, X = _X_all, labels = np.stack(_df_embed_joint['embedding_cluster_top_descriptor']), 
                                                                special_labels = ['Nutty $1^{st}$', 'Nutty $2^{nd}$', 'Nutty $3^{rd}$', 'Woody $1^{st}$', 'Woody $2^{nd}$','Woody $3^{rd}$', '_'], 
                                                                special_formatting = {'Nutty $1^{st}$' : {'marker' : 'o', 'annotations': [], 'color' : 126, 'size' : 1.3,'lw':0.5,'lc':(0,0,0,1)},
                                                                                        'Nutty $2^{nd}$' : {'marker' : 'o', 'annotations': [], 'color' : 50, 'size' : 1.3,'lw':0.5,'lc':(0,0,0,1)},
                                                                                        'Nutty $3^{rd}$' : {'marker' : 'o', 'annotations': [], 'color' : 0, 'size' : 1.3,'lw':0.5,'lc':(0,0,0,1)},
                                                                                        #'nutty_4' : {'marker' : 'o', 'annotations': [], 'color' : 160, 'size' : 0.7},
                                                                                        'Woody $1^{st}$' : {'marker' : 'o', 'annotations': [], 'color' : 130, 'size' : 1.3,'lw':0.5,'lc':(0,0,0,1)},
                                                                                        'Woody $2^{nd}$' : {'marker' : 'o', 'annotations': [], 'color' : 190, 'size' : 1.3,'lw':0.5,'lc':(0,0,0,1)},
                                                                                        'Woody $3^{rd}$' : {'marker' : 'o', 'annotations': [], 'color' : 220, 'size' : 1.3,'lw':0.5,'lc':(0,0,0,1)},
                                                                                        #'woody_4' : {'marker' : 'o', 'annotations': [], 'color' : 210, 'size' : 0.7},
                                                                                        '_' : {'marker' : 'o', 'annotations': [], 'color' : 258, 'size' : 0.3},
                                                                }, legend = True, cmap = mymap)


        



# Raw Data

In [9]:
# df_full = pd.read_csv('./Data/export_20220512-135815.csv', sep=';')
df_full = pd.read_csv('../Dataset/export_20220512-135815.csv', sep=';')
# df_uniprot = pd.read_csv('./Data/uniprot_sequences.csv', sep = ';', index_col = None)
df_uniprot = pd.read_csv('../Dataset/uniprot_sequences.csv', sep = ';', index_col = None)
df = pd.read_csv('./Data/predict_Model20220613-091215_e10000.csv', sep=';')
seq_input_df = pd.read_csv('./Data/seqs.csv', sep=';', index_col = None)
mol_input_df = pd.read_csv('./Data/mols.csv', sep=';', index_col = None)
seq_df_raw = pd.read_json('./Data/table_ncbi_identifier_name_sequence_stripped.json', orient='index')
df_odor = pd.read_csv('./Data/merged_GoodScent_merged_paper_SMILES_to_Smell.csv', sep=';')
df_odor_embedding_json = pd.read_json('./Data/pyrfume_embedding_2022_09_08_embed.json', orient = 'index').set_index('_SMILES')
df_odor_embedding_index = pd.read_csv('./Data/pyrfume_embedding_2022_09_08_source.csv', sep=';', index_col=0)
df_pyrfume = pd.read_csv('./Data/full_data.csv', sep=';', index_col = 0)
with open('./Data/map_CID_to_inchikey.json', 'r') as jsonfile:
    pyrfume_map_cid_to_inchikey = json.load(jsonfile)
with open('./Data/classes_mapping.json', 'r') as jsonfile:
    pyrfume_classes_mapping = json.load(jsonfile)
    pyrfume_classes_mapping = {val: key for key, val in pyrfume_classes_mapping.items()}
with open('./Data/map_inchikey_to_isomericSMILES.json', 'r') as jsonfile:
    map_inchikey_to_isomericSMILES = json.load(jsonfile)


df_full = df_full[df_full.Mixture != 'mixture']
df_uniprot.set_index('Entry', drop = True, inplace = True)
df_full = df_full.join(df_uniprot[['Uniprot_Sequence']], on = 'Uniprot ID', how = 'left')
df_full['_MolID'] = df_full.apply(lambda x: merge_cols_with_priority(x, primary_col = 'InChI Key', secondary_col = 'canonicalSMILES'), axis = 1)
df_full['_Sequence'] = df_full.apply(lambda x: merge_cols_with_priority(x, primary_col = 'Uniprot_Sequence', secondary_col = 'Sequence'), axis = 1)
df_full['mutated_Sequence'] = df_full.apply(lambda x: perform_mutation(x, mutation_col = 'Mutation', seq_col = '_Sequence'), axis = 1)
# Change units
df_full.loc[df_full['Unit'] == 'uM', 'Value'] = df_full[df_full['Unit'] == 'uM']['Value'].apply(lambda x: _change_units(x, _from = 'uM', _to = 'Log(M)'))
df_full.loc[df_full['Unit'] == 'uM', 'Unit'] = 'Log(M)'
df_full.loc[df_full['Unit_Screen'] == 'mM', 'Value_Screen'] = df_full[df_full['Unit_Screen'] == 'mM']['Value_Screen'].apply(lambda x: _change_units(x, _from = 'mM', _to = 'uM'))
df_full.loc[df_full['Unit_Screen'] == 'mM', 'Unit_Screen'] = 'uM'
# process EC50
data_ec50 = df_full[df_full['Parameter'] == 'ec50']
pairs_ec50 =_func_data_ec50(data_ec50)
# process Screening
data_screening = df_full[df_full['Parameter'] != 'ec50']
if not data_screening.empty:
        pairs_primary, pairs_secondary = _func_data_screening(data_screening)
        pairs_primary = pairs_primary.loc[pairs_primary.index.difference(pairs_ec50.index)] 
        pairs_secondary = pairs_secondary.loc[pairs_secondary.index.difference(pairs_ec50.index)]

        pairs = pd.concat([pairs_ec50, pairs_primary, pairs_secondary])
else:
    pairs = pairs_ec50
pairs['Responsive'] = pairs['Responsive'].astype(int)

# Merge sequence and molecule information.
_tmp_seq = pd.merge(seq_input_df, seq_df_raw, left_on = 'mutated_Sequence', right_on = 'Sequence')
_tmp_seq = _tmp_seq[['seq_id', 'Accesion_Number','Gene','Sequence']]
_tmp_pred = pd.merge(df, mol_input_df, on = '_SMILES')
df_predict = pd.merge(_tmp_pred, _tmp_seq, on='seq_id')
df_predict.set_index(['seq_id', 'mol_id'], drop=True, inplace = True)
df_predict.rename(columns = {'pred':'predict', '_MolID' : 'InchiKey'}, inplace = True)

df_predict_unstack = df_predict.copy()
df_predict_unstack.set_index(['Gene', 'InchiKey'], drop = True, inplace = True)
df_predict_unstack = df_predict_unstack['predict'].unstack(level = 0)

df_odor['SMILES'] = df_odor.apply(choose_smiles, axis=1)
df_odor.set_index('InchiKey', inplace = True)

# odor embedding:
df_odor_embedding = df_odor_embedding_index.join(df_odor_embedding_json, on = '_SMILES', how='inner')
df_odor_embedding.rename(columns = {'prediction' : 'odorModel_label', 'InChI Key' : 'InchiKey', '_SMILES' :'SMILES'}, inplace = True)
df_odor_embedding.drop(columns = ['_MolID', 'canonicalSMILES'], inplace = True)
df_odor_embedding.set_index('InchiKey', inplace = True)

# Map one-hot to labels:
df_pyrfume['pyrfume_odor'] = df_pyrfume['Values'].apply(lambda x: ','.join([pyrfume_classes_mapping[key] for key in list(np.where(np.array(x[1:-1].split(', '), dtype=int) == 1)[0])]))
df_pyrfume['InchiKey'] = df_pyrfume['CID'].astype(int).astype(str).map(pyrfume_map_cid_to_inchikey)

# Add GoodScent descriptors: 
df_odor = df_pyrfume.join(df_odor, on = 'InchiKey', how = 'left', rsuffix = '_GS')

# Mols view:
df_mols = df_predict_unstack.apply(lambda x: np.array(x), axis = 1)
df_mols.name = 'predict'
df_mols = df_mols.to_frame()
df_mols['SMILES'] = df_mols.index.map(map_inchikey_to_isomericSMILES)
df_mols['broadness'] = df_mols['predict'].apply(broadness_measure)

# OR view
df_OR = df_predict.groupby('Gene').apply(lambda x: np.array(x['predict']))
df_OR.name = 'predict'
df_OR = df_OR.to_frame()
df_OR['broadness'] = df_OR['predict'].apply(broadness_measure)


_tmp = df_mols[df_mols['broadness'] <= 10e-7].copy()
_tmp['max_predict'] = _tmp['predict'].apply(lambda x: np.max(x))
# Save non active SMILES
non_active_mols_idx = _tmp.index
del _tmp


df_mols_active = df_mols.loc[df_mols.index.difference(non_active_mols_idx)].copy()
df_mols_non_active = df_mols.loc[non_active_mols_idx].copy()

df_mols = df_mols_active.copy()
df_mols = df_mols[df_mols['SMILES'].apply(_test_CNOS_atomic_num)].copy()
df_odor = df_odor[~(df_odor['pyrfume_odor'].str.contains('odorless'))].copy()

_odor = df_odor.copy()
_odor = _odor.set_index('InchiKey')
_odor = _odor['pyrfume_odor'].dropna() 
_odor = df_mols.join(_odor, how = 'inner')

_odor_type = _odor.groupby('pyrfume_odor').count()['predict']
_odor = _odor[_odor['pyrfume_odor'].isin(_odor_type.index)] ## Pyrfume

# Generate clustering based on odor predcition embedding.
_data = np.stack(df_odor_embedding['embed'], axis = 0)
uamp_embedding = umap.UMAP(n_neighbors=15, min_dist=0.0, n_components=5, random_state=345).fit_transform(_data) # 15, 0
labels = hdbscan.HDBSCAN(min_samples=10, min_cluster_size=10).fit_predict(uamp_embedding) # 15 , 12 # 10, 10
df_odor_embedding['uamp_embedding'] = list(uamp_embedding)
df_odor_embedding['embedding_cluster_hdbscan'] = labels.astype(int)

_odor = df_mols.join(df_odor_embedding['embedding_cluster_hdbscan'], on = 'InchiKey', how='left')

_odor_type = _odor.groupby('embedding_cluster_hdbscan').count()['predict']
_odor = _odor[_odor['embedding_cluster_hdbscan'].isin(_odor_type.index)]

# Generate clustering based on odor predcition embedding.
df_code = df_mols.copy()
_data = np.stack(df_code['predict'], axis = 0)
uamp_embedding = umap.UMAP(n_neighbors=15, min_dist=0.0, n_components=5, random_state=345).fit_transform(_data) # 15, 0
labels = hdbscan.HDBSCAN(min_samples=10, min_cluster_size=10, metric = 'manhattan').fit_predict(uamp_embedding) # 15 , 12 # 10, 10
df_code['code_uamp_embedding'] = list(uamp_embedding)
df_code['code_cluster_hdbscan'] = labels.astype(int)
labels = AgglomerativeClustering(n_clusters = 100).fit_predict(_data)
df_code['code_cluster_aglo'] = labels.astype(int)

df_mols_joint = df_mols.copy()
df_mols_joint = df_mols_joint.join(df_odor_embedding[['embedding_cluster_hdbscan', 'odorModel_label', 'embed', 'uamp_embedding']], on = 'InchiKey', how='inner')
df_mols_joint = pd.merge(df_mols_joint, df_odor[['pyrfume_odor', 'GS_Odor type', 'merged_odors', 'InchiKey']], on = 'InchiKey', how='inner')
df_mols_joint = df_mols_joint.join(df_code[['code_cluster_hdbscan', 'code_uamp_embedding', 'code_cluster_aglo']], on = 'InchiKey', how='inner')
df_mols_joint.set_index('InchiKey', inplace = True)

embedding_cluster_top_descriptor_mapping = df_mols_joint.groupby('embedding_cluster_hdbscan').apply(lambda x: _get_most_occuring_pyrfume_descriptor(x, i=0, return_stats=False, top_k = 3)).to_dict()
embedding_cluster_top_descriptor_mapping.update({-1 : 'unclustered'})
df_mols_joint['embedding_cluster_top_descriptor'] = df_mols_joint['embedding_cluster_hdbscan'].map(embedding_cluster_top_descriptor_mapping)

/tmp/ipykernel_35824/3150660313.py:2: DtypeWarning: Columns (2,5,10,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df_full = pd.read_csv('../Dataset/export_20220512-135815.csv', sep=';')


INFO: To discard EC50 inconsistent: 17
INFO: To discard because of screening ordering (Inconsistent through Value_Screen): 144
INFO: To discard screening inconsistent per Value_Screen: 53
INFO: To discard screening inconsistent: 166


# 1 - Quantile Distribution (Figure 2)

In [ ]:
plt.rcParams['font.size'] = 36
_df = df_mols_joint.copy()
_df = _df[_df['embedding_cluster_hdbscan'] >= 0]
_temp=pd.DataFrame()
_temp['100%'] = _df.groupby('embedding_cluster_hdbscan').apply(get_quantile_per_odor_group, q = 1,)
_temp['90%'] = _df.groupby('embedding_cluster_hdbscan').apply(get_quantile_per_odor_group, q = 0.9,)
_temp['75%'] = _df.groupby('embedding_cluster_hdbscan').apply(get_quantile_per_odor_group, q = 0.75,)
_temp['50%'] = _df.groupby('embedding_cluster_hdbscan').apply(get_quantile_per_odor_group, q = 0.5,)

plt.figure(figsize=(15,8))
sns.histplot(data=_temp.melt(),x='value', hue='variable', cumulative=True, fill=False,
             bins=10000, alpha=1, element='step',palette='tab10', linewidth=2.5, stat='density', common_norm=False)
plt.xlabel(r"$Q_{k}(\alpha)$", labelpad=25)
plt.ylabel('Percentage of clusters',labelpad=25)
plt.xlim([0,350])
plt.xticks([0,50,100,150,200,250,300,350])
plt.legend(['50%','75%', '90%', '100%'],loc='lower right', title=r'$\alpha$', fontsize=26,title_fontsize=26)
plt.tight_layout(pad=0.5)
plt.savefig(f'./Figures/Quantile_Distribution_Cumulative_{date.today()}.png')

In [ ]:
plt.rcParams['font.size'] = 36
_df = df_mols_joint.copy()
_df = _df[_df['embedding_cluster_hdbscan'] >= 0]
_temp=pd.DataFrame()
_temp['100%'] = _df.groupby('embedding_cluster_hdbscan').apply(get_quantile_per_odor_group, q = 1,)
_temp['90%'] = _df.groupby('embedding_cluster_hdbscan').apply(get_quantile_per_odor_group, q = 0.9,)
_temp['75%'] = _df.groupby('embedding_cluster_hdbscan').apply(get_quantile_per_odor_group, q = 0.75,)
_temp['50%'] = _df.groupby('embedding_cluster_hdbscan').apply(get_quantile_per_odor_group, q = 0.5,)

plt.figure(figsize=(15,8))
sns.kdeplot(data=_temp.melt(),x='value', hue='variable',common_norm=False,
            alpha=1, palette='tab10', fill=False, bw_method='scott', bw_adjust=0.6, lw=3)
plt.xlabel(r"$Q_{k}(\alpha)$", labelpad=25)
plt.ylabel('Density', labelpad=25)
plt.xlim([0,350])
plt.xticks([0,50,100,150,200,250,300,350])
plt.legend(['50%','75%', '90%', '100%'],loc='upper right', title=r'$\alpha$',fontsize=26,title_fontsize=26)
plt.tight_layout(pad=0.5)
plt.savefig(f'./Figures/Quantile_Distribution_{date.today()}.png')

# 2 - Broadness Distribution (Figure 3)

In [ ]:
plt.rcParams['font.size'] = 30

df_predict['Responsive'] = df_predict.predict > 0.5
df_predict.reset_index(inplace=True)

_temp_dfs = []
for seq in df_predict.seq_id.unique():
    _temp_dfs.append(df_predict[(df_predict.seq_id == seq) & (df_predict.Responsive == 1)])

import tanimoto_similarity
pool = multiprocessing.Pool(processes=20)

mean_tanimoto = pool.map(tanimoto_similarity.get_tanimoto_matrix, _temp_dfs)
pool.close()
pool.join()

broadness_df = df_predict.groupby('seq_id').apply(lambda x: x.Responsive.sum() / x.Responsive.count()).reset_index().rename(columns={0:'Broadness'})
broadness_df = broadness_df.merge(df_predict.drop_duplicates(subset='seq_id')[['seq_id','Sequence','Gene']], on='seq_id')
broadness_df = broadness_df[['Sequence','seq_id','Broadness','Gene']]

order_dict = pd.Series(natsorted(broadness_df.Gene, alg=ns.IGNORECASE)).to_dict()
order_dict2 = {y: x for x, y in order_dict.items()}
broadness_df['natural order'] = broadness_df.Gene.map(order_dict2)
broadness_df['Tanimoto'] = mean_tanimoto

_broadness_df = broadness_df.sort_values('natural order')
_broadness_df['family'] = _broadness_df.Gene.apply(lambda x: get_family(x))
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(25,10),gridspec_kw={'height_ratios': [15, 1]})
fig.subplots_adjust(hspace=0.1)

copper = plt.get_cmap('copper_r')
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap
copper = truncate_colormap(copper, 0.1, 1)

rescale = lambda y: (y - np.min(y)) / (np.max(y) - np.min(y))
ax1.bar(_broadness_df['natural order'], height=_broadness_df['Broadness'], width=1., color=copper(rescale(_broadness_df.Tanimoto)))
a = mpl.cm.tab20

for i in broadness_df.sort_values('Broadness', ascending=False).head(10).index:
    ax1.text(_broadness_df.loc[i]['natural order'], _broadness_df.loc[i]['Broadness'], _broadness_df.loc[i]['Gene'], fontsize=19, rotation=70)

for i, fam in enumerate(_broadness_df.family.unique()):
    bar=ax2.barh(y = 0, left=_broadness_df[_broadness_df.family == fam]['natural order'].min(), width=len(_broadness_df[_broadness_df.family == fam]['natural order']), color=a.colors[i], height=0.5, align='center')
    if fam == '12' or fam == '3' :
        lab = ['']
    else:
        lab = [str(fam)]
    ax2.bar_label(bar, labels=lab, label_type='center',padding=0.0, fontsize=19,fontweight='bold')

plot = plt.scatter(_broadness_df['natural order'], _broadness_df['Broadness'], c = _broadness_df['Tanimoto'], cmap=copper, s=0)
cbaxes = inset_axes(ax1, width="30%", height="3%", loc=1)
plt.colorbar(plot, cax=cbaxes, orientation='horizontal',ticks=[0.1,0.15,0.2,0.25])
plt.xlabel('Mean Tanimoto similarity')


ax1.set_xticks([],[])
ax2.spines.bottom.set_visible(False)
ax2.spines.top.set_visible(False)
ax2.spines.left.set_visible(False)
ax2.spines.right.set_visible(False)
ax2.set_yticks([],[])
ax1.set_ylim([0,0.65])
ax2.set_xlabel('Olfactory receptors', labelpad=25)
ax1.set_ylabel('Broadness', labelpad=25)
plt.savefig(f'./Figures/Broadness_Distribution_{date.today()}.png',bbox_inches='tight',pad_inches = 0)

# 3 - Combinatorial Code Paper (Figure 4)

In [ ]:
gene_order = df_OR.sort_values('broadness', ascending=False).index
_df = df_mols_joint.copy()
_df = _df[_df['embedding_cluster_hdbscan'] >= 0]
_temp=pd.DataFrame()
_temp['100%'] = _df.groupby('embedding_cluster_hdbscan').apply(get_quantile_per_odor_group, q = 1,)
_temp['90%'] = _df.groupby('embedding_cluster_hdbscan').apply(get_quantile_per_odor_group, q = 0.9,)
_temp['75%'] = _df.groupby('embedding_cluster_hdbscan').apply(get_quantile_per_odor_group, q = 0.75,)
_temp['50%'] = _df.groupby('embedding_cluster_hdbscan').apply(get_quantile_per_odor_group, q = 0.5,)

for clus in [137, 64, 174, 3]: # [167, 38, 188, 4]
    cluster = clus
    mat_cluster = df_predict_unstack.loc[df_mols_joint[df_mols_joint.embedding_cluster_hdbscan == cluster].sort_values('broadness', ascending=False).index, gene_order]
    q50 = int(_temp.loc[cluster, '50%'])
    q75 = int(_temp.loc[cluster, '75%'])
    q90 = int(_temp.loc[cluster, '90%'])
    q100 = int(_temp.loc[cluster, '100%'])
    print(df_mols_joint[df_mols_joint.embedding_cluster_hdbscan == cluster].embedding_cluster_top_descriptor.unique())
    print(f"Q(a)({0.5, 0.75, 0.9, 1.0}) = {q50},{q75},{q90},{q100}")
    plt.figure(figsize=(15,10))
    plt.matshow(mat_cluster)
    plt.xticks([])
    plt.yticks([])
    #plt.savefig(f'./Figures/Combinatorial_Code_Cluster_{cluster}_{date.today()}.png',bbox_inches='tight',pad_inches = 0)
    plt.show()

# 4 - Umap Odor Embedding (Figure 5)

In [ ]:
# Generate clustering based on odor predcition embedding.
_df_embed = df_odor_embedding.copy()
_data = np.stack(_df_embed['embed'], axis = 0)
uamp_embedding = umap.UMAP(n_neighbors=15, min_dist=0.5, n_components=2, random_state=345).fit_transform(_data)
labels = hdbscan.HDBSCAN(min_samples=10, min_cluster_size=10).fit_predict(uamp_embedding) # 15 , 12
_df_embed['uamp_embedding'] = list(uamp_embedding)
_df_embed['embedding_cluster_hdbscan'] = labels.astype(int)

_df_embed = _df_embed.join(df_odor_embedding['embedding_cluster_hdbscan'], rsuffix = '_5D')

_df_embed_joint = _df_embed.join(df_mols_joint[['pyrfume_odor']], how = 'inner')
_df_embed_joint = _df_embed_joint.dropna(subset = ['pyrfume_odor'])
_embedding_cluster_top_descriptor_mapping = _df_embed_joint.groupby('embedding_cluster_hdbscan_5D').apply(lambda x: _get_most_occuring_pyrfume_descriptor(x, i=0, return_stats=False, top_k = 3)).to_dict()
_embedding_cluster_top_descriptor_mapping.update({-1 : '-1'})

_res = {}
j=0
for key, val in _embedding_cluster_top_descriptor_mapping.items():
    if len(val.split('_')) >1:
        if ('nutty' == val.split('_')[0]) or ('nut'==val.split('_')[0]):
            _res[key] = 'Nutty $1^{st}$'
        elif ('nutty' == val.split('_')[1]) or ('nut'==val.split('_')[1]):
            _res[key] = 'Nutty $2^{nd}$'
        elif ('nutty' == val.split('_')[2]) or ('nut'==val.split('_')[2]):
            _res[key] = 'Nutty $3^{rd}$'
        elif 'woody' in val.split('_')[0]:
            _res[key] = 'Woody $1^{st}$'
        elif 'woody' in val.split('_')[1]:
            _res[key] = 'Woody $2^{nd}$'
        elif 'woody' in val.split('_')[2]:
            _res[key] = 'Woody $3^{rd}$'
        else:
            _res[key] = '_'
    else:
        _res[key] = '_'
_embedding_cluster_top_descriptor_mapping = _res


_df_embed_joint['embedding_cluster_top_descriptor'] = _df_embed_joint['embedding_cluster_hdbscan_5D'].map(_embedding_cluster_top_descriptor_mapping)

_embedding_name = 'uamp_embedding' # 'pca_embedding' 'uamp_embedding'
plt.rcParams['font.size'] = 20
fig = plt.figure(figsize = (25, 16))
ax = fig.add_subplot(111)

_gene_names = ['OR2W1', 'OR10W1', 'OR5K1']
ORs_in_contour_plot(ax, _df_embed_joint, _embedding_name, _gene_names)
lgnd = ax.legend( prop={'size': 20})
for i in range(3,9):
    lgnd.legendHandles[i]._sizes = [200]
    
lgnd.legendHandles[0].set_linewidth(5)
lgnd.legendHandles[0].set_color('grey')
lgnd.legendHandles[1].set_linewidth(5)
lgnd.legendHandles[1].set_color('green')
lgnd.legendHandles[2].set_linewidth(5)
lgnd.legendHandles[2].set_color('blue')
plt.ylabel('')
plt.xlabel('')
plt.savefig(f'./Figures/Receptor_Odor_Umap_{date.today()}.png',bbox_inches='tight',pad_inches = 0)

# 5 - Olfactory Receptors vs Odors Distribution (Figure 6 - a/b)

## 1-1 By Activity

In [ ]:
plt.rcParams['font.size'] = 36
_pairs = pairs.reset_index()
non_responsive = pd.DataFrame(_pairs[_pairs.Responsive == 0].groupby(['_MolID']).count()['mutated_Sequence'])
responsive = pd.DataFrame(_pairs[_pairs.Responsive == 1].groupby(['_MolID']).count()['mutated_Sequence'])
repartition = non_responsive.merge(responsive, left_index=True, right_index=True,how='outer').replace(float('nan'),0)
repartition['total'] = repartition['mutated_Sequence_x'] + repartition['mutated_Sequence_y'] 

_repartition = repartition.sort_values('total', ascending=False)

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(15,10))
fig.subplots_adjust(hspace=0.05)

ax1.bar(_repartition.index, height=_repartition.mutated_Sequence_y, width=1., color='tab:red',edgecolor='black',linewidth=0.0)
ax1.bar(_repartition.index, bottom=_repartition.mutated_Sequence_y, height=_repartition.mutated_Sequence_x , width=1.,color='silver')

ax2.bar(_repartition.index, bottom=_repartition.mutated_Sequence_y, height=_repartition.mutated_Sequence_x , width=1.,color='silver')
ax2.bar(_repartition.index, height=_repartition.mutated_Sequence_y, width=1.,color='tab:red',edgecolor='black',linewidth=0.0)

ax1.set_ylim(310, 800)
ax2.set_ylim(0, 180)

ax1.spines.bottom.set_visible(False)
ax2.spines.top.set_visible(False)
ax1.xaxis.tick_top()
ax1.tick_params(labeltop=False)
ax2.xaxis.tick_bottom()
ax1.set_xticks([])
ax2.set_xticks([])
ax1.legend(['Agonists', 'Non-Agonists'])


d = 0.5
kwargs = dict(marker=[(-1, -d), (1, d)], markersize=12,
              linestyle="none", color='k', mec='k', mew=1, clip_on=False)
ax1.plot([0, 1], [0, 0], transform=ax1.transAxes, **kwargs)
ax2.plot([0, 1], [1, 1], transform=ax2.transAxes, **kwargs)

ax2.set_xlabel('Odorants', labelpad=25)
ax1.set_ylabel('Number of ORs tested', y=0.0001, labelpad=25)
plt.savefig(f'./Figures/Distribution_OR_Odor_by_activity_{date.today()}.png',bbox_inches='tight',pad_inches = 0)

## 1-2 By Tests

In [ ]:
plt.rcParams['font.size'] = 36
_pairs = pairs.reset_index()
ec50 = pd.DataFrame(_pairs[_pairs._DataQuality == 'ec50'].groupby(['_MolID']).count()['mutated_Sequence'])
secondary = pd.DataFrame(_pairs[_pairs._DataQuality == 'secondaryScreening'].groupby(['_MolID']).count()['mutated_Sequence'])
primary = pd.DataFrame(_pairs[_pairs._DataQuality == 'primaryScreening'].groupby(['_MolID']).count()['mutated_Sequence'])

repartition = ec50.merge(secondary, left_index=True, right_index=True,how='outer').merge(primary, left_index=True, right_index=True,how='outer').replace(float('nan'),0)
repartition['total'] = repartition['mutated_Sequence_x'] + repartition['mutated_Sequence_y'] + repartition['mutated_Sequence'] 

_repartition = repartition.sort_values('total', ascending=False)

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(15,10))
fig.subplots_adjust(hspace=0.05)

ax1.bar(_repartition.index, height=_repartition.mutated_Sequence_x, width=1.5, )
ax1.bar(_repartition.index, bottom=_repartition.mutated_Sequence_x, height=_repartition.mutated_Sequence_y , width=1.5, )
ax1.bar(_repartition.index, bottom=_repartition.mutated_Sequence_y +_repartition.mutated_Sequence_x, height=_repartition.mutated_Sequence , width=1.5, color='silver')

ax2.bar(_repartition.index, height=_repartition.mutated_Sequence_x, width=1.5, )
ax2.bar(_repartition.index, bottom=_repartition.mutated_Sequence_x, height=_repartition.mutated_Sequence_y , width=1.5, )
ax2.bar(_repartition.index, bottom=_repartition.mutated_Sequence_y +_repartition.mutated_Sequence_x, height=_repartition.mutated_Sequence , width=1.5,color='silver')

ax1.set_ylim(310, 800)
ax2.set_ylim(0, 180)

ax1.spines.bottom.set_visible(False)
ax2.spines.top.set_visible(False)
ax1.xaxis.tick_top()
ax1.tick_params(labeltop=False)
ax2.xaxis.tick_bottom()
ax1.set_xticks([])
ax2.set_xticks([])


ax1.legend(['EC50','Secondary','Primary'])
d = 1.5 
kwargs = dict(marker=[(-1, -d), (1, d)], markersize=12,
              linestyle="none", color='k', mec='k', mew=1, clip_on=False)
ax1.plot([0, 1], [0, 0], transform=ax1.transAxes, **kwargs)
ax2.plot([0, 1], [1, 1], transform=ax2.transAxes, **kwargs)

ax2.set_xlabel('Odorants', labelpad=25)
ax1.set_ylabel('Number of ORs tested', y=0.0001, labelpad=25)
plt.savefig(f'./Figures/Distribution_OR_Odor_by_test_{date.today()}.png',bbox_inches='tight',pad_inches = 0)

# 6 - Odors vs Olfactory Receptors Distribution (Figure 6 - c/d)

## 2-1 By Activity

In [ ]:
plt.rcParams['font.size'] = 36
_pairs = pairs.reset_index()
non_responsive = pd.DataFrame(_pairs[_pairs.Responsive == 0].groupby(['mutated_Sequence']).count()['_MolID'])
responsive = pd.DataFrame(_pairs[_pairs.Responsive == 1].groupby(['mutated_Sequence']).count()['_MolID'])
repartition = non_responsive.merge(responsive, left_index=True, right_index=True,how='outer').replace(float('nan'),0)

repartition['total'] = repartition['_MolID_x'] + repartition['_MolID_y'] 

_repartition = repartition.sort_values('total', ascending=False)

plt.figure(figsize=(15,10))
plt.bar(_repartition.index, height=_repartition._MolID_y, width=1, color='tab:red')
plt.bar(_repartition.index, bottom=_repartition._MolID_y, height=_repartition._MolID_x , width=1,color='silver')
plt.xticks([],[])
plt.legend(['Agonists','Non-Agonists'])
plt.xlabel('Olfactory receptors', labelpad=25)
plt.ylabel('Number of odorants tested',labelpad=25)
plt.savefig(f'./Figures/Distribution_Odor_OR_by_activity_{date.today()}.png',bbox_inches='tight',pad_inches = 0)

## 2-2 By Tests

In [ ]:
plt.rcParams['font.size'] = 36
_pairs = pairs.reset_index()
ec50 = pd.DataFrame(_pairs[_pairs._DataQuality == 'ec50'].groupby(['mutated_Sequence']).count()['_MolID'])
secondary = pd.DataFrame(_pairs[_pairs._DataQuality == 'secondaryScreening'].groupby(['mutated_Sequence']).count()['_MolID'])
primary = pd.DataFrame(_pairs[_pairs._DataQuality == 'primaryScreening'].groupby(['mutated_Sequence']).count()['_MolID'])

repartition = ec50.merge(secondary, left_index=True, right_index=True,how='outer').merge(primary, left_index=True, right_index=True,how='outer').replace(float('nan'),0)
repartition['total'] = repartition['_MolID_x'] + repartition['_MolID_y'] + repartition['_MolID'] 

_repartition = repartition.sort_values(['total','_MolID','_MolID_y','_MolID_x'], ascending=False)

plt.figure(figsize=(15,10))
plt.bar(_repartition.index, height=_repartition._MolID_x, width=1.5)
plt.bar(_repartition.index, bottom=_repartition._MolID_x, height=_repartition._MolID_y , width=1.5)
plt.bar(_repartition.index, bottom=_repartition._MolID_x + _repartition._MolID_y, height=_repartition._MolID , width=1.5, color='silver')
plt.xticks([],[])
plt.legend(['EC50','Secondary','Primary'])
plt.xlabel('Olfactory receptors', labelpad=25)
plt.ylabel('Number of odorants tested', labelpad=25)
plt.savefig(f'./Figures/Distribution_Odor_OR_by_test_{date.today()}.png',bbox_inches='tight',pad_inches = 0)

# 7 - Combinatorial Code Full

## 7 -1 Per Group

In [ ]:
gene_order = df_OR.sort_values('broadness', ascending=False).index
_df = df_mols_joint.copy()
_df = _df[_df['embedding_cluster_hdbscan'] >= 0]
_temp=pd.DataFrame()
_temp['100%'] = _df.groupby('embedding_cluster_hdbscan').apply(get_quantile_per_odor_group, q = 1,)
_temp['90%'] = _df.groupby('embedding_cluster_hdbscan').apply(get_quantile_per_odor_group, q = 0.9,)
_temp['75%'] = _df.groupby('embedding_cluster_hdbscan').apply(get_quantile_per_odor_group, q = 0.75,)
_temp['50%'] = _df.groupby('embedding_cluster_hdbscan').apply(get_quantile_per_odor_group, q = 0.5,)

for clus in df_mols_joint.embedding_cluster_hdbscan.unique():
    if clus >= 0: # Discarded unclustered molecules
        cluster = clus
        mat_cluster = df_predict_unstack.loc[df_mols_joint[df_mols_joint.embedding_cluster_hdbscan == cluster].sort_values('broadness', ascending=False).index, gene_order]
        q50  = _temp.loc[cluster, '50%']
        q75  = _temp.loc[cluster, '75%']
        q90  = _temp.loc[cluster, '90%']
        q100 = _temp.loc[cluster, '100%']
        print('{} : {}'.format(clus, df_mols_joint[df_mols_joint.embedding_cluster_hdbscan == cluster].embedding_cluster_top_descriptor.unique()))
        print(f"Q(a)({0.5, 0.75, 0.9, 1.0}) = {q50},{q75},{q90},{q100}")
        plt.figure(figsize=(15,10))
        plt.matshow(mat_cluster)
        plt.xticks([])
        plt.yticks([])
        #plt.savefig(f'./Figures/Combinatorial_Code_Cluster_{cluster}_{date.today()}.png',bbox_inches='tight',pad_inches = 0)
        plt.show()

## 7-2 Full

In [ ]:
_odor = df_odor.copy()
_odor = _odor.set_index('InchiKey')
_odor = _odor['GS_Odor type'].dropna()
_odor = df_mols.join(_odor, how = 'inner')

_odor_type = _odor.groupby('GS_Odor type').count()['predict']
_odor_type = _odor_type # [_odor_type > 20]
_odor = _odor[_odor['GS_Odor type'].isin(_odor_type.index)]

df_odor_OR = None
for smell in _odor_type.index:

    _df = df_predict_unstack.loc[_odor[_odor['GS_Odor type'] == smell].index].copy()
    _df = _df > 0.5
    _tmp = _df.sum(axis = 0) / len(_df) 
    _tmp.name = 'active_rate'
    _tmp.index.name = 'Gene'
    _tmp = _tmp.to_frame().join(df_OR['broadness'], how = 'left')
    _tmp['a/b'] = _tmp['active_rate'] / _tmp['broadness']
    _tmp = pd.concat([_tmp], keys=[smell], names=['Odor'])
    if df_odor_OR is None:
        df_odor_OR = _tmp.copy()
    else:
        df_odor_OR = pd.concat([df_odor_OR,_tmp])

_mols_order_by = 'embedding_cluster_top_descriptor' 
_gene_order_by = 'broadness' 
# --- Molecules ----------------------------------------------
def get_ordered_mols(df, _mols_order_by, _df_predict_unstack, discard_unclustered = True):
    _mols = df.loc[_df_predict_unstack.index.intersection(df.index)].copy()
    _mols = _mols.dropna(subset = [_mols_order_by])
    if _mols_order_by in ['embedding_cluster_hdbscan', 'embedding_cluster_top_descriptor']:
        if discard_unclustered:
            _mols = _mols[_mols['embedding_cluster_hdbscan'] >= 0]
        if _mols_order_by == 'embedding_cluster_top_descriptor':
            _mols['embedding_cluster_top_descriptor'] = _mols.apply(lambda x: str(x['embedding_cluster_top_descriptor']) + '_' + str(x['embedding_cluster_hdbscan']) , axis = 1)
        _mols[_mols_order_by] = _mols[_mols_order_by].astype(str)

        
    _mols = _mols.sort_values([_mols_order_by, 'broadness'])
    _mols = _mols[_mols_order_by]
    return _mols

# --- Receptors ----------------------------------------------
def get_ordered_genes(df, _gene_order_by):
    """
    df_OR
    df_odor_OR
    """
    if _gene_order_by == 'broadness':
        _gene = df.copy()
        _gene = _gene.sort_values('broadness')
    elif _gene_order_by == 'family':
        _gene = df.copy()
        _gene['family'] = _gene.index.str.extract(pat = r'(\d+)').values 
        _gene = _gene.sort_values(_gene_order_by)
    elif _gene_order_by == 'family_broadness':
        _gene = df.copy()
        _gene['family'] = _gene.index.str.extract(pat = r'(\d+)').values 
        _gene = _gene.sort_values(['family', 'broadness'])
    else: #  _gene_order_by in df.index.get_level_values('Odor').unique():
        _gene = df.copy()
        _gene = _gene.loc[_gene_order_by, slice(None)].sort_values('a/b', ascending = False)    
    return _gene


_mols = get_ordered_mols(df_mols_joint, _mols_order_by, df_predict_unstack)
if _gene_order_by in df_odor_OR.index.get_level_values('Odor').unique():
    _gene = get_ordered_genes(df_odor_OR, _gene_order_by)
else:
    _gene = get_ordered_genes(df_OR, _gene_order_by)
_gene_idx = _gene.index
# --- Plot ----------------------------------------------
def _tmp_func(x, _discard_idx):
    if x in _discard_idx:
        return ''
    else:
        return x

_tmp = _mols.reset_index().groupby(by = _mols_order_by, sort = True).count()['InchiKey']
_xticks = _tmp.cumsum() - 1
_xticks = _xticks.reset_index()
_discard_idx = _tmp[_tmp < 10].index
_xticks[_mols_order_by] = _xticks[_mols_order_by].apply(lambda x: _tmp_func(x, _discard_idx))
del _tmp, _discard_idx
_tmp = df_predict_unstack.loc[_mols.index][_gene_idx]

plt.rcParams['font.size']=2

fig = plt.figure(dpi = 300, figsize = [15, 10])
ax = fig.add_subplot(111)
cax = ax.matshow(_tmp.T.values)

ax.set_xticks(list(_xticks['InchiKey']), list(_xticks[_mols_order_by]), fontsize=2, rotation=90) # fontsize = 3
ax.xaxis.set_tick_params(width=0.1)

if _gene_order_by in ['family', 'family_broadness']:
    _tmp = _gene.reset_index().groupby(by = 'family', sort = True).count()['Gene']
    _yticks = _tmp.cumsum() - 1
    _yticks = _yticks.reset_index()
    ax.set_yticklabels(list(_yticks['Gene']), list(_yticks['family']), fontsize = 2) # fontsize = 2
    ax.yaxis.set_tick_params(width=0.1, rotation = 90)
    ax.set_yticks(list(range(len(_gene_idx))),_gene_idx.to_list(), fontsize=2)
    ax.yaxis.set_tick_params(width=0.1)
    del _tmp
else:
    ax.set_yticks(list(range(len(_gene_idx))),_gene_idx.to_list())
    ax.set_yticks([],[])

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

plt.show() 